# DST80 crack_Toyota

Auteur: [Kaci Amaouche](mailto:amaouchekaci28@gmail.com)

Dans ce [Jupyter](https://jupyter.org/) notebook,nous présentons une attaque sur DST80 pour Toyota. Si vous n'êtes pas familier avec Jupyter, vous pouvez jeter un coup d'œil rapide à la documentation ou aux tutoriels disponibles[Notebook Basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html) guide (~5min).

Ce notebook comporte:

* [Configuration de l'environnement]
* [Implémentation]

## 1 - Configuration de l'environnement

Pour pouvoir exécuter ces scripts, il va falloir:
1. Ce notebook (the `.ipynb` file) et le script DST80_Algo qui doit être dans le même répertoire
1. Python >= 3.8
1. Seules les librairies standard sont utilisées 

## 2 - Implémentation

# Une brève explication

Selon les informations fournies dans le document de cryptanalyse, il n'est pas nécessaire d'entreprendre une cryptanalyse pour le modèle Toyota. Il vous suffira de récupérer les pages 1, 2 et 3 du transpondeur, ainsi que l'ID et trois constantes de 5 octets.

 Une fois ces éléments en votre possession, vous pourrez rapidement reconstituer la clé en utilisant des relations linéaires quasi instantanées.

D'après la notation des auteurs, $S[0]$ est l'octet de poids faible, donc en implémentant le pseudo-code, tout sera inversé, par exemple, $S_2[0]$ représente en réalité $S_2[4]$

Voici le pseudo-code de l'attaque


In [48]:
#**Algorithm 4: Toyota Key Generation Algorithm**
#*Reverse engineered from immobiliser firmware. The three 5-byte constants X0...X2 have been redacted on the manufacturer's request.*

#1. Initialize X as [X0, X1, X2]
#2. **function gen_key(page1, page2, id)**
#3.   keyL ← (id << 16) | (page1 << 8) | page2
#4.   keyR ← keyL
#5.   for i in {0...7} do
#6.     S1 ← keyR
#7.     S2 ← keyR ⊕ X[i mod 3]
#8.     S3[0] ← ((S2[0] + S2[2] + S2[3]) & 0xff) << 19
#9.     S3[1] ← ((S2[2] + S2[3] + S2[4]) & 0xff) << 10
#10.    S3[2] ← ((S2[0] + S2[1] + S2[3]) & 0xff) << 11
#11.    S3[3] ← ((S2[0] + S2[1] + S2[4]) & 0xff) << 12
#12.    S3[4] ← ((S2[1] + S2[2] + S2[4]) & 0xff) << 13
#13.    keyR = S3 ⊕ keyL
#14.    keyL = S3
#15.   end for
#16. return keyL, keyR


Une façcon d'implémenter :

In [51]:
def generate_key(page1, page2, ID, X0, X1, X2):

    # On définit ces variables selon le pseudo-code
    X = [X0, X1, X2]
    keyL = (ID << 16) | (page1 << 8) | page2
    keyR = keyL
    
    for i in range(8):
        # On définit S1 et S2
        S1 = keyR
        S2 = keyR ^ X[2-(i % 3)]

        #On converit S2 en liste
        S2_list = [(S2 >> (i * 8)) & 0xFF for i in range(4, -1, -1)]

        #On déclare une liste vide S3
        S3=[0]*5

        #On définit les élements de S3 selon le pseudo-code mais en mode inversé, car S3[0] représente le byte de poids faible, et donc on inverse seulement selon ce schéma :
        # S2[0] ==>S2[4]
        # S2[1] ==>S2[3]
        # S2[2] ==>S2[2]
        # S2[3] ==>S2[1]
        # S2[4] ==>S2[0]

        S3[4]= ((S2_list[4] + S2_list[2] + S2_list[1] ) & 0xFF ) << 1
        S3[3]= ((S2_list[2] + S2_list[1] + S2_list[0] ) & 0xFF ) << 3
        S3[2]= ((S2_list[4] + S2_list[3] + S2_list[1] ) & 0xFF ) << 1
        S3[1]= ((S2_list[4] + S2_list[3] + S2_list[0] ) & 0xFF ) << 3
        S3[0]= ((S2_list[3] + S2_list[2] + S2_list[0] ) & 0xFF ) << 1

        #On convertit KeyL en liste
        keyL_list = [(keyL >> (i * 8)) & 0xFF for i in range(4, -1, -1)]

        #On contruit keyR selon le pseudo-code
        keyR=[S3[i] ^ keyL_list[i] for i in range(5)]

        #On convertit keyR en entier
        keyR=keyR[4] + keyR[3]*16**2 + keyR[2]*16**4 + keyR[1]*16**6 + keyR[0]*16**8
        keyL=S1

    return keyL, keyR


Exemple d'exécution

In [52]:
page1, page2, ID, X0, X1, X2= 0x1234567, 0x7456123, 0x321478, 0x9845124, 0xef125cd4, 0xabc14523e
generate_key(page1, page2, ID, X0, X1, X2)

(2142938069729, 1151359959121)